In [1]:
from typing import List
from flask import Flask, request, jsonify
import threading
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime

app = Flask(__name__)

# Define database model
Base = declarative_base()

class LogEntry(Base):
    __tablename__ = 'log_entries'
    id = Column(Integer, primary_key=True)
    matrix = Column(String)
    response = Column(String)
    request_time = Column(DateTime)
    response_time = Column(DateTime)

# Function to compute the largest rectangle
def largest_rectangle(matrix: List[List[int]]) -> tuple:
    if not matrix or not matrix[0]:
        raise ValueError("Invalid matrix")
    rows = len(matrix)
    cols = len(matrix[0])
    max_area = 0
    max_number = 0
    for row in range(rows):
        for col in range(cols):
            if matrix[row][col] != -9: # Skip processed cells
                current_number = matrix[row][col]
                current_area = 0
                # Find the maximum area rectangle with the current number
                for i in range(row, rows):
                    if matrix[i][col] == current_number:
                        width = 0
                        while col + width < cols and matrix[i][col + width] == current_number:
                            width += 1
                        height = i - row + 1
                        current_area = max(current_area, width * height)
                if current_area > max_area:
                    max_area = current_area
                    max_number = current_number
    return max_number, max_area

# Route for welcome message
@app.route('/')
def welcome():
    return "Welcome to the Largest Rectangle API!"

# Route for largest rectangle computation
@app.route('/largest-rectangle', methods=['POST'])
def get_largest_rectangle():
    try:
        matrix = request.json.get('matrix')
        if not matrix:
            raise ValueError("Matrix not provided in the request")
        request_time = datetime.now()
        largest_number, largest_rectangle_area = largest_rectangle(matrix)
        response_time = datetime.now()

        # Save entry to the database
        engine = create_engine("sqlite:///log.db")
        Base.metadata.create_all(engine) # Create tables if not exists
        Session = sessionmaker(bind=engine)
        session = Session()
        new_entry = LogEntry(matrix=str(matrix), response=str({"largest_number": largest_number, "largest_rectangle_area": largest_rectangle_area}), request_time=request_time, response_time=response_time)
        session.add(new_entry)
        session.commit()
        session.close()

        return jsonify({"largest_number": largest_number, "largest_rectangle_area": largest_rectangle_area})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

def run_flask_app():
    app.run(debug=True, host='127.0.0.1', port=5000, use_reloader=False)

if __name__ == '__main__':
    threading.Thread(target=run_flask_app).start()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jan/2024 13:06:26] "POST /largest-rectangle HTTP/1.1" 200 -
